In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
%run train.py --clearml-run-name latent_8_dim_beta_10_v3 --batch-size 128 --test-batch-size 256 --epochs 301 \
    --lr 0.001 --latent-dim 8 --model-name latent_8_dim_beta_10 --beta 10

ClearML Task: created new task id=dd5fe0f4628a4a759d8f82a0d50c0f6b
2023-10-31 12:01:10,108 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/49f472236d7240f4bd330a30202378ad/experiments/dd5fe0f4628a4a759d8f82a0d50c0f6b/output/log


  0%|          | 0/301 [00:00<?, ?it/s]

2023-10-31 12:01:28,707 - clearml.frameworks - INFO - Found existing registered model id=b1013fc5defc47bda93f72291becbc59 [/home/quczer/repos/latent-geometry/models/mnist/latent_8_dim_beta_10_encoder.pt] reusing it.
2023-10-31 12:01:38,459 - clearml.frameworks - INFO - Found existing registered model id=aef5501861304682b664da4f9868bd8c [/home/quczer/repos/latent-geometry/models/mnist/latent_8_dim_beta_10_decoder.pt] reusing it.


100%|██████████| 301/301 [50:34<00:00, 10.08s/it]  
